In [2]:
import pandas as pd
import numpy as np
import os,sys
sys.path.append(os.path.realpath('..'))
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV
import joblib
import datautil
from sklearn.metrics import classification_report, confusion_matrix
import pickle as pkl
from joblib import dump, load


In [9]:
full_data = pd.read_csv('../train.csv')
full_data = full_data[(full_data['adr'] < 1000) & (full_data['adr'] > -100)] # remove outliers

# get the preprocessor and the default training features
preprocessor, features_spec = datautil.get_the_data_preprocessor()

# split data into input and labeled
X_train_full_raw = full_data[features_spec]
y = np.array(full_data['is_canceled'])
X = preprocessor.fit_transform(X_train_full_raw)

In [5]:
# implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=66)

In [7]:
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

# predictions
rfc_predict = rfc.predict(X_test)

In [ ]:
# score
rfc_cv_score = cross_val_score(rfc, X, y, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

In [ ]:
# randomized search
param_grid = {
    'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [int(x) for x in np.linspace(100, 500, num = 11)] + [None]
}

gsearch1 = RandomizedSearchCV(estimator = RandomForestClassifier(), param_distributions = param_grid, verbose=3, scoring='roc_auc', cv=3, n_iter=100, random_state=42, n_jobs=-1)
gsearch1.fit(X,y)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

# save grid search 
dump(gsearch1.best_estimator_, 'randomforest.model')

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


In [11]:
from xgboost import XGBClassifier
rfc = XGBClassifier()
rfc.fit(X_train,y_train)

# predictions
rfc_predict = rfc.predict(X_test)

# score
rfc_cv_score = cross_val_score(rfc, X, y, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - XGBoost: ", rfc_cv_score.mean())

C:\Users\tina\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:09:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    9.9s finished


=== Confusion Matrix ===
[[14692]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14692

    accuracy                           1.00     14692
   macro avg       1.00      1.00      1.00     14692
weighted avg       1.00      1.00      1.00     14692



=== All AUC Scores ===
[0.8776161  0.85318443 0.834735  ]


=== Mean AUC Score ===
Mean AUC Score - XGBoost:  0.8551785100446946


In [10]:
from lightgbm import LGBMClassifier
rfc = LGBMClassifier()
rfc.fit(X_train,y_train)

# predictions
rfc_predict = rfc.predict(X_test)

# score
rfc_cv_score = cross_val_score(rfc, X, y, cv=3, scoring='roc_auc', n_jobs=-1, verbose=3)

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - LightGBM: ", rfc_cv_score.mean())

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.3s finished


=== Confusion Matrix ===
[[14692]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14692

    accuracy                           1.00     14692
   macro avg       1.00      1.00      1.00     14692
weighted avg       1.00      1.00      1.00     14692



=== All AUC Scores ===
[0.88563735 0.86931078 0.84427889]


=== Mean AUC Score ===
Mean AUC Score - LightGBM:  0.8664090063696371


In [28]:
dump(rfc, 'lightgbm.model')

['lightgbm.model']